In [0]:
from pyspark.sql.utils import AnalysisException
from delta.tables import DeltaTable

def table_exists(catalog_name, db_name, table_name):
    try:
        spark.sql(f"DESCRIBE TABLE {catalog_name}.{db_name}.{table_name}")
        return True
    except AnalysisException:
        return False

def merge_delta_data(input_df, catalog_name, db_name, table_name, folder_path, merge_condition, partition_column):
    spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
    full_table_name = f"{catalog_name}.{db_name}.{table_name}"

    if table_exists(catalog_name, db_name, table_name):
        deltaTable = DeltaTable.forName(spark, full_table_name)
        update_columns = [c for c in input_df.columns if c != partition_column]
        deltaTable.alias("tgt") \
            .merge(input_df.alias("src"), merge_condition) \
            .whenMatchedUpdate(
                set = {c: f"src.{c}" for c in update_columns}
            ) \
            .whenNotMatchedInsertAll() \
            .execute()
    else:
        input_df.write \
            .mode("overwrite") \
            .format("delta") \
            .partitionBy(partition_column) \
            .option("path", f"{folder_path}/{table_name}") \
            .saveAsTable(full_table_name)